# Sea ice and ocean data access and analysis 

Accessing coincident sea ice and ocean data to study melt pond characteristics.

Intro to use case, motivation, connections to cloud migration, learning objectives...

## Explore NASA Earthdata sea ice and ocean products...

... highlight key search terms and data availability across NASA DAACs...

## Import packages

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pprint import pprint
import getpass
import requests
import json
import random
from statistics import mean


## Data Discovery

Start by identifying your study area and exploring coincident data over the same time and area.

NASA Earthdata Search can be used to visualize file coverage over mulitple data sets and to access the same data you will be working with below: [insert URL with same search here]


### Identify time and area of interest


Study area: Using OA melt pond annotation as an example: https://openaltimetry.org/data/icesat2/?start_date=2019-06-22&annoId=180

- Temporal coverage: 22 June 2019
- Bounding area: -62.8,81.7,-56.4,83

In [4]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-62.8,81.7,-56.4,83'
# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2019-06-22T00:00:00Z,2019-06-22T23:59:59Z'

### Explore data availability using the Common Metadata Repository
The Common Metadata Repository (CMR) is a high-performance, high-quality, continuously evolving metadata system that catalogs Earth Science data and associated service metadata records. These metadata records are registered, modified, discovered, and accessed through programmatic interfaces leveraging standard protocols and APIs. Note that not all NSIDC data can be searched at the file level using CMR, particularly those outside of the NASA DAAC program.

General CMR API documentation: https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html

[below is background for development - will not include in final notebook]

#### Configure a CMR GraphQL client
Using qgl we can communicate with the CMR GraphQL endpoint in a standards-based way, allowing for schema introspection. gql isn't the only python GraphQL client library out there. Other libraries might provide features you value, like gql-next's static type generation functionality.

GraphQL endpoint documentation and interactive playground: https://graphql.earthdata.nasa.gov/api

In [3]:
CMR_GRAPHQL_URL = 'https://graphql.earthdata.nasa.gov/api'
sample_transport=RequestsHTTPTransport(
    url=CMR_GRAPHQL_URL,
    retries=3,            # Automatically retry, don't put it on the user!
)

client = Client(
    transport=sample_transport,
    fetch_schema_from_transport=True,  # Get the schema as part of the client object
)

collection_schema = client.schema.get_type('Collection')

# Show info about 5 random fields
sample_fields = random.sample(list(collection_schema.fields.items()), 5)
for fieldname, field in sample_fields:
    print(f'* {fieldname}: {field.description}')

* ancillaryKeywords: Allows authors to provide words or phrases outside of the controlled Science Keyword vocabulary, to further describe the collection.
* originalFormat: Original metadata format.
* timeEnd: End time of the metadata
* metadataDates: Dates related to activities involving the metadata record itself.
* relatedUrls: Describes any data/service related URLs that include project home pages, services, related data archives/servers, metadata extensions, direct links to online software packages, web mapping services, links to images, or other data.


### Select data sets and determine version numbers

Data products:
- Sea Surface Temperature: 
    * MODIS (Terra) SST: MODIS_T-JPL-L2P-v2014.0
    * SMAP SSS L3 expected in cloud Ops starting with 20.4.2.
    * GRACE-FO in cloud Ops now.
- Sea Ice Temperature?
- Height data?

Data sets are selected by data set IDs (e.g. ATL07). In the CMR API documentation, a data set ids is referred to as a "short name". These short names are located at the top of each NSIDC data set landing page in gray above the full title. We are using the Python Requests package to access the CMR. Data are then converted to JSON format; a language independant human-readable open-standard file format. More than one version can exist for a given data set:

In [5]:
query = gql('''
query { 
  collections(shortName: "MOD29") {
    items { 
      shortName
      datasetId
      conceptId
      versionId
    }
  }
}
''')

# As a user, I don't need to know about "feed" or "entry", just the fields I'm interested in!
response = client.execute(query)
pprint(response)

{'collections': {'items': [{'conceptId': 'C1219248592-LANCEMODIS',
                            'datasetId': 'MODIS/Terra Near Real Time (NRT) Sea '
                                         'Ice Extent 5-Min L2 Swath 1km',
                            'shortName': 'MOD29',
                            'versionId': '6NRT'},
                           {'conceptId': 'C61468238-NSIDC_ECS',
                            'datasetId': 'MODIS/Terra Sea Ice Extent 5-Min L2 '
                                         'Swath 1km V005',
                            'shortName': 'MOD29',
                            'versionId': '5'},
                           {'conceptId': 'C1000001160-NSIDC_ECS',
                            'datasetId': 'MODIS/Terra Sea Ice Extent 5-Min L2 '
                                         'Swath 1km V006',
                            'shortName': 'MOD29',
                            'versionId': '6'}]}}


We will specify the most recent version for our remaining data set queries.

### Create data search dictionary

Create a nested dictionary with each data set shortname and version, as well as shared temporal range and polygonal area of interest. Data set shortnames, or IDs, as well as version numbers, are located at the top of every NSIDC landing page.

***Note that some version IDs include 3 digits and others include only 1 digit. Make sure to enter this value exactly as reported above.***

In [9]:
data_search = { 'SST': {'short_name': 'MOD29','version': '6','bounding_box': bounding_box,'temporal':temporal},
                'xxx': {'short_name': 'xxx','version': '1','bounding_box': bounding_box,'temporal':temporal},
             'xxx': {'short_name': 'xxx','version': '1','bounding_box': bounding_box,'temporal':temporal}
            }

### Determine how many files exist over this time and area of interest, as well as the average size and total volume of those granules
We will use the gql library once more, this time to query the CMR granule API. We will look at the results and print the number of granules, average size, and total volume of those granules.

Finally, we update the data_dict with the granule count. TODO: Is this necessary?

In [10]:
# TODO: Get rid of data_dict entirely? or re-use it here? Currently hardcoding bbox, temporal.
# NOTE: GraphQL endpoint currently supports selecting granules by conceptId, not short_name, versionId.
query = gql('''
query {
  granules(conceptId: "C1000001160-NSIDC_ECS"
           boundingBox: "-62.8,81.7,-56.4,83"
           temporal: "2019-06-22T00:00:00Z,2019-06-22T23:59:59Z"
           limit: 100) {
    count
    items { granuleSize }
           
  }
}
''')
response = client.execute(query)

# Now that it's a so easy to get the info we need, I think granule_info can go away.
granule_sizes = [float(i['granuleSize']) for i in response['granules']['items']][:]

print(f"Found {response['granules']['count']} granules")
print(f"Average size: {mean(granule_sizes):.2f}")
print(f"Total size: {sum(granule_sizes):.2f}")

dataset_info['gran_num'] = int(response['granules']['count'])

Found 10 granules
Average size: 1.84
Total size: 18.37


Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.



## Data Access

Depending on the NSIDC data set, we could utilize CMR UMM-S associations to demonstrate ESI subsetting service availability. Or for "native" data access, we could also access using Harmony (all CMR data can be accessed using Harmony no-proc service), or just through CMR data access URLs. 

The PO DAAC data can be subsetted via Harmony subsetting service.

## Data Comparison

- Any reprojection or resampling needed? 
- Simple plotting of data 